In [1]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas as pd
import pandas_ta as ta
import yfinance as yf

end_date = '2024-05-20'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*10)

raw_df = yf.download("QQQ",
                 start=start_date,
                 end=end_date)

raw_df

[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,QQQ,QQQ,QQQ,QQQ,QQQ,QQQ
Date,,,,,,
2014-05-23 00:00:00+00:00,82.584770,89.879997,89.900002,89.120003,89.320000,22687300
2014-05-27 00:00:00+00:00,83.613876,91.000000,91.019997,90.199997,90.279999,29725200
2014-05-28 00:00:00+00:00,83.356606,90.720001,91.099998,90.650002,90.970001,30468100
2014-05-29 00:00:00+00:00,83.889511,91.300003,91.309998,90.860001,91.040001,33036100
2014-05-30 00:00:00+00:00,83.898720,91.309998,91.449997,90.830002,91.320000,29906400
...,...,...,...,...,...,...
2024-05-13 00:00:00+00:00,441.756073,443.079987,444.089996,441.649994,443.989990,22994200
2024-05-14 00:00:00+00:00,444.597595,445.929993,446.470001,442.459991,442.649994,34478300


In [5]:
df = raw_df.assign(prev_close=raw_df['Close'].shift(), high_minus_low=raw_df['High']-raw_df['Low'])
df

Price,Adj Close,Close,High,Low,Open,Volume,prev_close,high_minus_low
Ticker,QQQ,QQQ,QQQ,QQQ,QQQ,QQQ,,
Date,,,,,,,,
1999-03-10 00:00:00+00:00,43.417294,51.062500,51.156250,50.281250,51.125000,5232000,NaN,0.875000
1999-03-11 00:00:00+00:00,43.629841,51.312500,51.734375,50.312500,51.437500,9688600,51.062500,1.421875
1999-03-12 00:00:00+00:00,42.567017,50.062500,51.156250,49.656250,51.125000,8743600,51.312500,1.500000
1999-03-15 00:00:00+00:00,43.789272,51.500000,51.562500,49.906250,50.437500,6369000,50.062500,1.656250
1999-03-16 00:00:00+00:00,44.161278,51.937500,52.156250,51.156250,51.718750,4905800,51.500000,1.000000
...,...,...,...,...,...,...,...,...
2024-06-25 00:00:00+00:00,478.707306,479.380005,479.679993,475.119995,476.070007,29250200,473.959991,4.559998
2024-06-26 00:00:00+00:00,479.695892,480.369995,480.920013,478.130005,478.549988,22737100,479.380005,2.790009


In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas as pd
import pandas_ta as ta
import yfinance as yf

end_date = '2024-07-02'
start_date = '1999-03-10'

raw_df = yf.download("QQQ",
                 start=start_date,
                 end=end_date)

# Get prev high into df
df = raw_df.assign(prev_high=raw_df['High'].shift(), high_minus_low=raw_df['High']-raw_df['Low'])


def calc_ibs(data):
    # Compute the IBS indicator: (Close - Low) / (High - Low);
    return (data.Close - data.Low)/(data.High - data.Low)

def calc_rolling_mean(arr, n):
     # Compute the rolling mean of High minus Low over the last 25 days;
    return pd.Series(arr['High'] - arr['Low']).rolling(n).mean()  

def calc_rolling_high(arr, n):
    return pd.Series(arr['High']).rolling(n).max()

def calc_lower_band(arr):
    # Compute a lower band as the rolling High over the last 10 days minus 2.5 x the rolling mean of High minus Low (first bullet);
    return pd.Series(arr['High']).rolling(10).max() - (pd.Series(arr['high_minus_low']).rolling(25).mean() * 2.5)


class RsiOscillator(Strategy):
    upper_bound = 70
    lower_bound = 30
    def init(self):
        # Compute the rolling mean of High minus Low over the last 25 days;
        self.rolling_mean = self.I(calc_rolling_mean, self.data, 25)
        # Compute the IBS indicator: (Close - Low) / (High - Low);
        self.ibs = self.I(calc_ibs, self.data)
        # Compute a lower band as the rolling High over the last 10 days minus 2.5 x the rolling mean of High minus Low (first bullet);
        self.rolling_high = self.I(calc_rolling_high, self.data, 10)
        self.prev_high = self.data.prev_high
        self.lower_band = self.I(calc_lower_band, self.data)
    def next(self):
        # Go long whenever SPY closes under the lower band (3rd bullet), and IBS is lower than 0.3;
        if (self.data.Close < self.lower_band) and self.ibs < 0.3:
            self.buy()
        # Close the trade whenever the SPY close is higher than yesterday's high.
        if self.data.Close > self.data.prev_high:
            print(f'sell ${self.data.Close}')
            self.position.close()

bt =Backtest(df, RsiOscillator, cash = 100_000)
stats = bt.run()
bt.plot()
stats

[*********************100%***********************]  1 of 1 completed


TypeError: other must be a MultiIndex or a list of tuples